In [4]:
import time, json
import pandas as pd
from bs4 import BeautifulSoup
import requests
import csv
import requests, re

imported


In [5]:
with open('AED_locations_webscrape.html', 'r', encoding='utf-8') as file:
    html = file.read()


In [6]:
soup = BeautifulSoup(html, 'html.parser')

# We need to find all script tags
script_tags = soup.find_all('script')

data = []

# reg.ex pattern to extract markerOpts data
pattern = re.compile(r"markerOpts\['id'\] = (\d+);[\s\S]*?markerOpts\['markerLat'\] = ([\d.]+);[\s\S]*?markerOpts\['markerLng'\] = ([\d.]+);[\s\S]*?markerOpts\['bubbleHtml'\] = '([^']+)'")

for script_tag in script_tags:
    script_text = script_tag.string
    if script_text:
        matches = pattern.findall(script_text)
        for match in matches:
            marker_opts = {}
            marker_opts['id'] = float(match[0])
            marker_opts['markerLat'] = float(match[1])
            marker_opts['markerLng'] = float(match[2])
            bubble_html = match[3]

            # Extract next level layer of bubble html
            bubble_soup = BeautifulSoup(bubble_html, 'html.parser')
            spans = bubble_soup.find_all('span')
            for span in spans:
                span_text = span.get_text(strip=True)
                if ':' in span_text:
                    key, value = map(str.strip, span_text.split(':', 1))
                    # Correct the case of the keys to match fieldnames
                    key = key.capitalize()
                    #print(key)
                    # fix the typos for the key.
                    if key == 'Plassering hjertestartarter' or key == 'Plassering':
                        key = 'Plassering hjertestarter'
                    if key == 'Åpnignstider':
                        key = 'Åpningstider'
                    if key in ['Bransje', 'Åpningstider', 'Plassering hjertestarter']:
                      marker_opts[key] = value
                    else:
                        # Handle unexpected field names in a sep. col.
                        marker_opts['Unexpected Field Names'] = marker_opts.get('Unexpected Field Names', '') + f"{key}: {value}\n"


            # We fill missing data with null.
            for key in ['Bransje', 'Åpningstider', 'Plassering hjertestarter', 'Unexpected Field Names']:
                if key not in marker_opts:
                    marker_opts[key] = "null"

            data.append(marker_opts)

# Save it to a csv.
csv_filename = 'AED_facility_locations.csv'
with open(csv_filename, 'w', newline='') as csvfile:
    fieldnames = ['id', 'markerLat', 'markerLng', 'Bransje', 'Åpningstider', 'Plassering hjertestarter', 'Unexpected Field Names']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in data:
        writer.writerow(row)

print(f'Data extracted and saved to {csv_filename}')


Data extracted and saved to marker_data.csv


Data extracted and saved to marker_data.csv
